In [1]:
import numpy as np
import pandas as pd
import numba
from numba import cuda
import math
import time
from google.colab.patches import cv2_imshow
import cv2

In [2]:
!wget https://github.com/MyCatsGitHub/HPC_labs/raw/main/Salt%20and%20Pepper/chad.bmp \
        https://github.com/MyCatsGitHub/HPC_labs/raw/main/Salt%20and%20Pepper/mood.bmp \
        https://github.com/MyCatsGitHub/HPC_labs/raw/main/Salt%20and%20Pepper/peppe.bmp \
        https://raw.githubusercontent.com/MyCatsGitHub/HPC_labs/main/Bilateral%20filtering/sharikov.bmp

--2023-12-21 09:53:14--  https://github.com/MyCatsGitHub/HPC_labs/raw/main/Salt%20and%20Pepper/chad.bmp
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MyCatsGitHub/HPC_labs/main/Salt%20and%20Pepper/chad.bmp [following]
--2023-12-21 09:53:15--  https://raw.githubusercontent.com/MyCatsGitHub/HPC_labs/main/Salt%20and%20Pepper/chad.bmp
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6220938 (5.9M) [application/octet-stream]
Saving to: ‘chad.bmp.1’

chad.bmp.1          100%[===================>]   5.93M  --.-KB/s    in 0.05s   

2023-12-21 09:53:15 (109 MB/s) - ‘chad.bmp.1’ saved [6220938/6220938]



In [3]:
def add_noise(image, border):
    rand_noise = np.random.rand(image.shape[0], image.shape[1])
    noise_image = image.copy()
    noise_image[rand_noise < border] = 0
    noise_image[rand_noise > 1 - border] = 255
    return noise_image

In [4]:
def cpu_median_filter(image):
    filtered_image = image.copy()
    for x in range(1,len(image) - 1):
        for y in range(1, len(image[x]) - 1):
            tmp = np.zeros(9)

            tmp[0] = image[x-1, y-1]
            tmp[1] = image[x-1, y]
            tmp[2] = image[x-1, y+1]
            tmp[3] = image[x, y-1]
            tmp[4] = image[x, y]
            tmp[5] = image[x,y+1]
            tmp[6] = image[x+1,y-1]
            tmp[7] = image[x+1,y]
            tmp[8] = image[x+1,y+1]

            for w in range(8):
                for h in range(8-w):
                    if tmp[h] > tmp[h+1]:
                        tmp[h],tmp[h+1] = tmp[h+1], tmp[h]
            filtered_image[x,y] = tmp[int(len(tmp)/2)]
    return filtered_image

In [5]:
@cuda.jit
def gpu_median_filter(image, filtered_image):
    i,j = cuda.grid(2)
    tmp = cuda.local.array(shape=9,dtype=numba.int64)
    tmp[0] = image[i-1, j-1]
    tmp[1] = image[i-1, j]
    tmp[2] = image[i-1, j+1]
    tmp[3] = image[i, j-1]
    tmp[4] = image[i, j]
    tmp[5] = image[i,j+1]
    tmp[6] = image[i+1,j-1]
    tmp[7] = image[i+1,j]
    tmp[8] = image[i+1,j+1]

    for w in range(8):
        for h in range(8-w):
            if tmp[h] > tmp[h+1]:
                tmp[h], tmp[h+1] = tmp[h+1], tmp[h]
    filtered_image[i,j] = tmp[int(len(tmp)/2)]

In [6]:
def gpu_processing(image):
    threads_per_block = (16,16)
    blocks_per_grid = (math.ceil(image.shape[0]/threads_per_block[0]), math.ceil(image.shape[1]/ threads_per_block[1]))

    image_cuda = cuda.to_device(image)
    filtered_image_cuda = cuda.to_device(image.copy())

    gpu_median_filter[blocks_per_grid, threads_per_block](image_cuda, filtered_image_cuda)

    return filtered_image_cuda.copy_to_host()

In [12]:
def experiment():
    border = 0.1
    df = pd.DataFrame(columns = ['image shape','cpu time','cpu cv2 time','gpu time','boost native','boost cv2'])

    images = [(cv2.imread(name,cv2.IMREAD_GRAYSCALE),name) for name in ['mood.bmp','sharikov.bmp','chad.bmp','peppe.bmp']]
    shapes = []
    cpu_times = []
    cpu_cv2_times = []
    gpu_times = []
    boost = []
    boost_cv2 = []

    for img,name in images:
        shapes.append(str(img.shape[0]) + '*' +str(img.shape[1]))
        noised = add_noise(img,border)

        start_cpu = time.time()
        cpu_image = cpu_median_filter(noised)
        cpu_end = time.time() - start_cpu
        cpu_times.append(cpu_end)

        gpu_start = time.time()
        gpu_image = gpu_processing(noised)
        gpu_end = time.time() - gpu_start
        gpu_times.append(gpu_end)

        boost.append(cpu_end/ gpu_end)

        start_cv2_blur = time.time()
        cv2.medianBlur(noised,3)

        cv2_blur_end = time.time() - start_cv2_blur
        cpu_cv2_times.append(cv2_blur_end)

        boost_cv2.append(cv2_blur_end / gpu_end)

        print('Original')
        cv2_imshow(img)
        print('noised')
        cv2_imshow(noised)
        print('Filtered')
        cv2_imshow(gpu_image)

        cv2.imwrite('filtered ' + name, gpu_image)
    df['image shape'] = shapes
    df['cpu time'] = cpu_times
    df['cpu cv2 time'] = cpu_cv2_times
    df['gpu time'] = gpu_times
    df['boost native'] = boost
    df['boost cv2'] = boost_cv2

    return df

In [13]:
data = experiment()

Output hidden; open in https://colab.research.google.com to view.

In [14]:
data

,image shape,cpu time,cpu cv2 time,gpu time,boost native,boost cv2
0,240*320,1.389930,0.000230,0.002239,620.718697,0.102853
1,630*930,12.472454,0.000214,0.002021,6172.656519,0.105959
2,1440*1440,43.003010,0.002424,0.004462,9638.115742,0.543176
3,2400*2400,96.934703,0.004345,0.008301,11677.780647,0.523466
